In [1]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import pandas as pd
import os
os.environ["TOKENIZERS_PARALLELISM"] = "1"

from config import Config
from fliker_comment_tokenizer import FlikerCommentTokenizer
from pathlib import Path
from transformers import AutoTokenizer
from tqdm import tqdm


/opt/anaconda3/envs/torch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
config = Config()

fliker_comment_toeknizer = FlikerCommentTokenizer.train_tokenizer(config=config)
fliker_comment_toeknizer

Enriched img id: /tmp/enriched_results.csv
tokens: 128000
tokenizer.is_fast: True


100%|██████████| 159/159 [00:00<00:00, 580.55it/s]





Saved new fliker comment tokenizer at: /Users/chengbai/ml/cheng_git/notebooks/paligemma-3b-mix-224-tokenizer


GemmaTokenizerFast(name_or_path='google/paligemma-3b-mix-224', vocab_size=128000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<bos>', 'eos_token': '<eos>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<image>']}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<eos>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("<bos>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("<image>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [7]:
new_fliker_comment_toeknizer = FlikerCommentTokenizer.get_tokenizer(config=config)

# test_data = "hello, 會 員"
test_data = "<image><bos>cheng bai<pad><eos>"
new_fliker_comment_toeknizer.decode(new_fliker_comment_toeknizer.encode(test_data))

tokens: 128000
tokenizer.is_fast: True


'<bos><image><bos>cheng bai<pad><eos>'

In [8]:

print(f"<image>: {new_fliker_comment_toeknizer.encode('<image>')}")
print(f"<bos>: {new_fliker_comment_toeknizer.encode('<bos>')}")
print(f"<eos>: {new_fliker_comment_toeknizer.encode('<eos>')}")
print(f"<pad>: {new_fliker_comment_toeknizer.encode('<pad>')}")
print(f"cheng bai: {new_fliker_comment_toeknizer.encode('cheng bai')}")

<image>: [2, 4]
<bos>: [2, 2]
<eos>: [2, 1]
<pad>: [2, 0]
cheng bai: [2, 6473, 144, 183, 68]


In [9]:
new_fliker_comment_toeknizer.decode(0)


'<pad>'

In [10]:
dir(new_fliker_comment_toeknizer)

['SPECIAL_TOKENS_ATTRIBUTES',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_bos_token',
 '_add_eos_token',
 '_add_tokens',
 '_additional_special_tokens',
 '_auto_class',
 '_batch_encode_plus',
 '_bos_token',
 '_call_one',
 '_cls_token',
 '_compile_jinja_template',
 '_convert_encoding',
 '_convert_id_to_token',
 '_convert_token_to_id_with_added_voc',
 '_create_repo',
 '_decode',
 '_decode_use_source_tokenizer',
 '_encode_plus',
 '_eos_token',
 '_eventual_warn_about_too_long_sequence',
 '_eventually_correct_t5_max_length',
 '_from_pretrained',
 '_get_files_timestamps',
 '_get_padding_truncation_strateg

In [18]:
new_fliker_comment_toeknizer.vocab_size, len(new_fliker_comment_toeknizer.get_vocab()), 
new_fliker_comment_toeknizer.get_added_vocab()

for k, v in new_fliker_comment_toeknizer.vocab.items():
    if v ==0:
        print(k)
        break


<pad>
